# Rastreamento de objetos e machine learning

Rastreamento de cores, objetos e aplicação de técnicas de machine learning pré-treinadas e desenvolvimento de algoritmos próprios.

Alguns recursos e códigos foram adaptados deste [repositório](https://github.com/udacity/CVND_Exercises/) do curso de Visão Computacional da Udacity.

> Atenção: este notebook foi desenhado para funcionar no **Google Collab**. Se pretende executar localmente prefira a versão local deste notebook, sem o sufixo ```-collab```.

## 1. Requerimentos

### 1.1 Bibliotecas

* OpenCV>=3.4.3
* Pillow>= 7.0.0
* Pytorch>=1.4.0
* Numpy>=1.18.1

### 1.2 Arquivos

Baixe o repositório do GitHub utilizando o comando abaixo. Em caso de atualização, utilize o comando para apagar o diretório antes.

In [ ]:
!rm -rf fiap-ml-visao-computacional/

In [ ]:
!git clone https://github.com/michelpf/fiap-ml-visao-computacional

Vamos agora posicionar o diretório do repositório para a aula respectiva. Nesse caso envie o comando de mudança de diretório.

In [ ]:
%cd fiap-ml-visao-computacional/aula-5-machine-learning-aplicado/

Importação das bibliotecas.

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

#Exibição na mesma tela do Jupyter
%matplotlib inline

from io import BytesIO
from IPython.display import clear_output, Image, display
import PIL

import datetime

from os import listdir
from os.path import isfile, join

plt.style.use('seaborn')
sns.set_style("whitegrid", {'axes.grid' : False})

## 2. Machine Learning

### 2.1 Reconhecimento Facial

Gerando 100 exemplos de faces, utilizando a câmera.

In [ ]:
# Extrator de faces
def face_extractor(imagem):
    classificador_face = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')
    imagem_gray = cv2.cvtColor(imagem,cv2.COLOR_RGB2GRAY)
    faces = classificador_face.detectMultiScale(imagem_gray, 1.2, 5)
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        face_recortada = imagem[y:y+h, x:x+w]

    return face_recortada

#### Extração de características

Gerando imagems de exemplos para ser posteriormente treinado. Neste caso vamos adotar um tamanho de imagem para processamento de 200 x 200 (empírico).

In [ ]:
imagem = cv2.imread("imagens/camera_output_1.png")
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(20,10))
plt.imshow(imagem)
plt.title("Extração de faces")

Vamos utilizar o diretório ```imagens/faces/michel``` que contem imagens de treino de uma pessoa (eu 😁).

#### Treinamento do modelo

Podemos testar os diversos tipos de classificadores, no entanto, o classificador LBPH tem o uso com melhor performance dentre o Eingenfaces e Fisherfaces.
Neste caso como temos apenas uma única pessoa, nosso dicionário de pessoas, ficou apenas com um único registro. Em casos de multiclasses, ou seja, mais de uma pessoa, cada uma delas deve ter um *id* associado, que é o valor de chave do dicionário.

In [ ]:
# Carregando exemplos de arquivos previamente coletados
faces_path = 'imagens/faces/michel/'
lista_arquivos_imagens = [f for f in listdir(faces_path) if isfile(join(faces_path, f))]

dados_treinamento, labels = [], []

# Lendo as imagens e associando a um label
for i, arquivos in enumerate(lista_arquivos_imagens):
    imagem_path = faces_path + lista_arquivos_imagens[i]
    imagem = cv2.imread(imagem_path, cv2.IMREAD_GRAYSCALE)
    dados_treinamento.append(imagem)
    labels.append(0)

# Criando uma matriz da lista de labels
labels = np.asarray(labels, dtype=np.int32)

# Treinamento do modelo
model = cv2.face.LBPHFaceRecognizer_create()

model.train(dados_treinamento, labels)

print("Modelo treinado com sucesso.")

pessoas = {0: "Michel"}

#### Inferência do modelo

Função para identificar o rosto e segmentar da imagme principal. Também utilizaremos para desenhar um retângulo delimitador.
Note que estamos normalizando a imagem (mesma escala, 200 x 200) que as imagens de treinamento.

In [ ]:
def face_detector(imagem):
    classificador_face = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')
    imagem_gray = cv2.cvtColor(imagem,cv2.COLOR_RGB2GRAY)
    faces = classificador_face.detectMultiScale(imagem_gray, 1.1, 5)
    
    if faces is ():
        return imagem, [], 0, 0
    
    for (x,y,w,h) in faces:
        cv2.rectangle(imagem,(x,y),(x+w,y+h),(0,255,255),2)
        roi = imagem[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    
    return imagem, roi, x, y

Obtendo as imagems por meio da câmera e fazendo a inferência on-line.

In [ ]:
imagem = cv2.imread("imagens/analise-face-michel.PNG")
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
imagem, face, x, y = face_detector(imagem)

if face is not ():
    face = cv2.cvtColor(face, cv2.COLOR_RGB2GRAY)
    predicao = model.predict(face)

    if x > 0:
        notificacao = "Dist. " + str(int(predicao[1])) + ' ' + pessoas[predicao[0]] 
        cv2.putText(imagem, notificacao, (x, y-20), cv2.FONT_HERSHEY_DUPLEX, 1, (255,120,150), 2)

    if int(predicao[1]) < 40:
        cv2.putText(imagem, "Reconhecido com sucesso", (x, y-50), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 2)
    else:
        cv2.putText(imagem, "Nao reconhecido", (250, 450), cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,255), 2)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(imagem)
plt.title("Inferência do modelo")

### 2.2 Classificador de Objetos
#### Yolo

É necessário baixar os pesos (modelo de deep-learning) neste link https://pjreddie.com/media/files/yolov3.weights e copiar para  pasta weights. O comando a seguir vai baixar o arquivo de pesos no diretório ```pesos```.

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights -P pesos/

In [ ]:
from utils import *
from darknet import Darknet

# Configurações na rede neural YOLOv3
cfg_file = 'cfg/yolov3.cfg'
m = Darknet(cfg_file)

# Pesos pré-treinados
weight_file = 'pesos/yolov3.weights'
m.load_weights(weight_file)

# Rótulos de classes
namesfile = 'data/coco.names'
class_names = load_class_names(namesfile)

In [ ]:
# Topologia da rede neural da YOLOv3
m.print_network()

In [ ]:
print("Tamanho da imagem de entrada da rede: " + str(m.width) + "x" + str(m.height))

In [ ]:
# Tamanho da figura
plt.rcParams['figure.figsize'] = [24.0, 14.0]

# Carregando imagem para classificar
img_path = "imagens/camara.jpg"
img = cv2.imread(img_path)

# Convertendo para o espaço de cores RGB
original_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Redimensionando imagem para ser compatível com a primeira camada da rede neural  
resized_image = cv2.resize(original_image, (m.width, m.height))

# Exibição das imagens
plt.subplot(121)
plt.title("Imagem Original")
plt.imshow(original_image)
plt.subplot(122)
plt.title("Imagem Redimensionada")
plt.imshow(resized_image)
plt.show()

In [ ]:
# Patamar de NMS (Non-Maximum Supression)
# Ajuste de sensibilidade de imagens com baixa luminosidade
nms_thresh = 0.6

# Patamar do IOU (Intersect of Union), indicador se o retângulo 
# de identificação de imagem foi adequadamente desenhado
iou_thresh = 0.4

In [ ]:
# Definindo tamnaho do gráfico
plt.rcParams['figure.figsize'] = [24.0, 14.0]

# Carregar imagem para classificação
img = cv2.imread(img_path)

# Conversão para o espaço RGB
original_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Redimensionamento para adatapção da primeira camada da rede neural 
resized_image = cv2.resize(original_image, (m.width, m.height))

# Deteteção de objetos na imagem
boxes = detect_objects(m, resized_image, iou_thresh, nms_thresh)

# Objetos encontrados e nível de confiança
print_objects(boxes, class_names)

# Desenho no gráfico com os regângulos e rótulos
plot_boxes(original_image, boxes, class_names, plot_labels = True)

In [ ]:
list_objects(boxes, class_names)